<a href="https://colab.research.google.com/github/ingabLee/Transformers_book/blob/main/TransformerLearning_Chap91_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia

import wikipedia

keyword = "Moderna"

#set lang en
wikipedia.set_lang("en")

# search keyword and display output
search_response = wikipedia.search(keyword)
print(search_response)

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=a4d110678a2b4a960aabb79f6ad3320cc9b61fc9f390457ede57205ae222f7ce
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
['Moderna', 'Moderna COVID-19 vaccine', 'Studio Moderna', 'Moderna de Pueblo', 'Moderna Museet', 'La Moderna', 'La Mujer Moderna', 'Moderna University', 'Museo MAGA', "Galleria d'Arte Moderna"]
